In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="17rFuCNZUUY1xHrMq1WTamV-JWh_IDZe8", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/04_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Imports Device
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_05_imports_device.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Exploring the Latent Space

*Part 4 of the Vizuara series on Variational Autoencoders From Scratch*
*Estimated time: 45 minutes*

## 1. Why Does This Matter?

We have built the machine. We derived the ELBO, implemented the reparameterization trick, and trained a working VAE on MNIST. The loss went down, the reconstructions looked reasonable.

But here is the thing: **training loss is just a number.** The real question is: *what has the model actually learned?*

In this notebook, we are going to open the hood and look inside. Specifically, we will:

1. **Visualize the latent space** -- see how the VAE organizes different digits into a coherent 2D map
2. **Generate brand-new digits** -- sample random points from the latent space and watch the decoder turn them into handwritten digits that never existed before
3. **Morph between digits** -- smoothly interpolate from a 3 to an 8 and see every intermediate shape along the way
4. **Map the entire manifold** -- create a sweeping grid over the 2D latent space and decode every point, producing one of the most beautiful visualizations in all of machine learning

This is the payoff. This is where everything clicks.

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Think of the latent space as a **map**. Not a map of a physical place, but a map of *digit styles*.

Every location on this map corresponds to a specific type of handwritten digit. If you walk north on the map, maybe the digits get rounder. Walk east, and maybe they develop sharper strokes. Walk from the "3 region" to the "8 region," and you see a smooth, continuous transition -- first the 3 grows a top loop, then fills in, until it becomes an 8.

This is the VAE's superpower. A regular autoencoder would scatter digits randomly across the space, with dead zones in between that decode to garbage. But the VAE, thanks to the KL divergence term, is forced to organize everything smoothly around the origin. **Every** point on the map decodes to something meaningful.

Let us verify this claim with actual experiments.

In [ ]:
#@title 🎧 Transition: Setup Vae
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_04_setup_vae.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. Setup -- Building and Training the VAE

Since each notebook in this series must run independently in Google Colab, we will include the complete VAE model code and train it from scratch on MNIST. This takes roughly 2-3 minutes on a Colab T4 GPU.

If you have already completed Notebook 3, the code below will look familiar -- it is the same architecture and loss function.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"Device: {device}")

In [ ]:
#@title 🎧 Listen: Model Definition Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_06_model_definition_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 3.1 Model Definition

We use a 2-dimensional latent space so that we can directly visualize it as a scatter plot. The encoder outputs $\mu$ and $\log(\sigma^2)$ for each input, and the decoder maps a 2D latent code back to a 784-dimensional pixel-probability vector.

In [ ]:
#@title 🎧 Code Walkthrough: Model Definition Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_07_model_definition_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=512, latent_dim=2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 256)
        self.fc_mu = nn.Linear(256, latent_dim)
        self.fc_logvar = nn.Linear(256, latent_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar


class Decoder(nn.Module):
    def __init__(self, latent_dim=2, hidden_dim=512, output_dim=784):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim, 256)
        self.fc2 = nn.Linear(256, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return torch.sigmoid(self.fc3(h))


class VAE(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=512, latent_dim=2):
        super().__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + std * eps

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar


def vae_loss(x_recon, x, mu, logvar):
    """ELBO loss = reconstruction (BCE) + KL divergence."""
    recon_loss = F.binary_cross_entropy(x_recon, x, reduction='sum')
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_loss


print("Model classes defined.")

In [ ]:
#@title 🎧 Transition: Data Loading Training Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_08_data_loading_training_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 3.2 Data Loading and Training

In [ ]:
#@title 🎧 Code Walkthrough: Training Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_09_training_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Load MNIST
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

# Initialize model
model = VAE(latent_dim=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
print("Training VAE on MNIST (20 epochs)...")
print("-" * 40)

model.train()
for epoch in range(20):
    total_loss = 0
    for data, _ in train_loader:
        data = data.view(-1, 784).to(device)
        optimizer.zero_grad()
        x_recon, mu, logvar = model(data)
        loss = vae_loss(x_recon, data, mu, logvar)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader.dataset)
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"  Epoch {epoch+1:2d}/20  |  Loss: {avg_loss:.2f}")

print("-" * 40)
print("Training complete!")

In [ ]:
#@title 🎧 Listen: Sanity Check Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_10_sanity_check_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us do a quick sanity check before diving into exploration. We will reconstruct a few test images to confirm the model is working properly.

In [ ]:
#@title 🎧 What to Look For: Sanity Check Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_11_sanity_check_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Quick sanity check: show original vs reconstruction
model.eval()
with torch.no_grad():
    sample_data, _ = next(iter(test_loader))
    sample_data = sample_data[:8].view(-1, 784).to(device)
    recon, _, _ = model(sample_data)

fig, axes = plt.subplots(2, 8, figsize=(14, 3.5))
for i in range(8):
    axes[0, i].imshow(sample_data[i].cpu().view(28, 28), cmap='gray')
    axes[0, i].axis('off')
    axes[1, i].imshow(recon[i].cpu().view(28, 28), cmap='gray')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Original', fontsize=12, rotation=0, labelpad=60)
axes[1, 0].set_ylabel('Reconstructed', fontsize=12, rotation=0, labelpad=60)
fig.suptitle('Sanity Check: Original vs. Reconstructed Digits', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
print("Reconstructions look reasonable -- we are ready to explore!")

In [ ]:
#@title 🎧 Transition: Explore Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_12_explore_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let Us Explore -- Component by Component

Now for the fun part. We have a trained VAE with a 2-dimensional latent space. Let us systematically explore what this model has learned.

In [ ]:
#@title 🎧 Listen: Latent Scatter Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_13_latent_scatter_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.1 Latent Space Scatter Plot

The first thing we want to know is: **how does the VAE organize different digits in latent space?**

We will pass every image in the 10,000-image test set through the encoder, extract the mean $\mu$ (ignoring the variance for visualization purposes), and plot each point colored by its true digit label.

If the VAE has learned a meaningful representation, we should see digits of the same class clustering together, and digits that look visually similar (like 4 and 9, or 3 and 8) placed near each other.

In [ ]:
#@title 🎧 Code Walkthrough: Encode Test Set
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_14_encode_test_set.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Encode the entire test set
model.eval()
all_mu = []
all_labels = []

with torch.no_grad():
    for data, labels in test_loader:
        data = data.view(-1, 784).to(device)
        mu, _ = model.encoder(data)
        all_mu.append(mu.cpu())
        all_labels.append(labels)

all_mu = torch.cat(all_mu, dim=0).numpy()
all_labels = torch.cat(all_labels, dim=0).numpy()

print(f"Encoded {len(all_mu)} test images into 2D latent space.")
print(f"Latent code range: [{all_mu.min():.2f}, {all_mu.max():.2f}]")

In [ ]:
#@title 🎧 What to Look For: Latent Scatter Plot Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_15_latent_scatter_plot_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Scatter plot of the latent space
fig, ax = plt.subplots(figsize=(10, 8))

scatter = ax.scatter(
    all_mu[:, 0], all_mu[:, 1],
    c=all_labels, cmap='tab10',
    s=3, alpha=0.5, edgecolors='none'
)

cbar = plt.colorbar(scatter, ax=ax, ticks=range(10))
cbar.set_label('Digit Class', fontsize=12)
cbar.ax.set_yticklabels([str(i) for i in range(10)])

ax.set_xlabel('$z_1$', fontsize=14)
ax.set_ylabel('$z_2$', fontsize=14)
ax.set_title('VAE Latent Space — 10,000 Test Digits', fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Latent Scatter Explanation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_16_latent_scatter_explanation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Take a moment to study this plot carefully. Notice several things:

- **Each digit class forms a recognizable cluster.** The 0s live in one region, the 1s in another, the 7s in yet another.
- **Visually similar digits are placed close together.** Look for 4 and 9 -- they often overlap, which makes sense because a sloppy 4 can look very much like a 9. Similarly, 3 and 8 tend to be neighbors.
- **The overall distribution is roughly centered at the origin** and approximately Gaussian-shaped. This is the KL divergence doing its job -- it pushes all encoder distributions toward $\mathcal{N}(0, I)$.
- **There are smooth transitions between clusters**, not hard boundaries. This is what makes generation and interpolation work.

This is a *structured, meaningful* latent space. Proximity implies similarity.

In [ ]:
#@title 🎧 Listen: Generate Digits Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_17_generate_digits_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Generating New Digits

Now for generation. The beauty of a VAE is that we can create digits that have never existed before. The recipe is simple:

1. Sample a random vector $z \sim \mathcal{N}(0, I)$
2. Pass it through the decoder
3. Out comes a new digit

Because the KL term trained the encoder to place latent codes near $\mathcal{N}(0, I)$, random samples from this distribution will land in meaningful regions of the latent space.

In [ ]:
#@title 🎧 What to Look For: Generate Digits Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_18_generate_digits_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Generate new digits from random latent codes
model.eval()
n_samples = 25  # 5x5 grid

with torch.no_grad():
    # Sample z from standard normal
    z_random = torch.randn(n_samples, 2).to(device)
    generated = model.decoder(z_random).cpu().view(-1, 28, 28)

fig, axes = plt.subplots(5, 5, figsize=(8, 8))
for i in range(5):
    for j in range(5):
        idx = i * 5 + j
        axes[i, j].imshow(generated[idx], cmap='gray')
        axes[i, j].axis('off')

fig.suptitle('Generated Digits — Sampled from $z \\sim \\mathcal{N}(0, I)$',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Each of these digits was generated from a random 2D latent code.")
print("They never existed in the training set -- the VAE invented them.")

In [ ]:
#@title 🎧 Listen: Generate Digits Explanation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_19_generate_digits_explanation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Look at the diversity. Some digits are cleaner than others, and some might be ambiguous -- is that a 4 or a 9? But the remarkable thing is that they are all *plausible* handwritten digits, synthesized from nothing but random noise passed through a learned decoder.

This is the power of a well-organized latent space. **Any** random point decodes to something meaningful.

In [ ]:
#@title 🎧 Listen: Interpolation Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_20_interpolation_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Latent Space Interpolation

This is one of the most elegant demonstrations in generative modeling. We will take two specific digits -- a **3** and an **8** -- encode them into latent space, and then walk along the straight line connecting them, decoding every step along the way.

If the latent space is truly smooth and continuous, we should see a gradual transformation with no sudden jumps, no garbage, and no dead zones.

In [ ]:
#@title 🎧 Code Walkthrough: Find Digits Encode
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_21_find_digits_encode.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Find a nice 3 and a nice 8 from the test set
model.eval()

# Get a specific 3 and 8
digit_3_img = None
digit_8_img = None

for data, labels in test_loader:
    for i in range(len(labels)):
        if labels[i] == 3 and digit_3_img is None:
            digit_3_img = data[i]
        if labels[i] == 8 and digit_8_img is None:
            digit_8_img = data[i]
        if digit_3_img is not None and digit_8_img is not None:
            break
    if digit_3_img is not None and digit_8_img is not None:
        break

# Encode both digits
with torch.no_grad():
    x_3 = digit_3_img.view(1, 784).to(device)
    x_8 = digit_8_img.view(1, 784).to(device)
    mu_3, _ = model.encoder(x_3)
    mu_8, _ = model.encoder(x_8)

print(f"Latent code for digit 3: z = [{mu_3[0, 0]:.3f}, {mu_3[0, 1]:.3f}]")
print(f"Latent code for digit 8: z = [{mu_8[0, 0]:.3f}, {mu_8[0, 1]:.3f}]")

In [ ]:
#@title 🎧 What to Look For: Interpolation Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_22_interpolation_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Interpolate between the two latent codes
n_steps = 12

with torch.no_grad():
    # Linear interpolation: z(t) = (1-t)*z_3 + t*z_8
    alphas = torch.linspace(0, 1, n_steps)
    z_interp = torch.stack([
        (1 - a) * mu_3 + a * mu_8 for a in alphas
    ]).squeeze(1).to(device)

    # Decode every interpolated point
    interp_images = model.decoder(z_interp).cpu().view(-1, 28, 28)

# Display as a filmstrip
fig, axes = plt.subplots(1, n_steps, figsize=(18, 2.5))
for i in range(n_steps):
    axes[i].imshow(interp_images[i], cmap='gray')
    axes[i].axis('off')
    if i == 0:
        axes[i].set_title('3', fontsize=13, fontweight='bold', color='#2196F3')
    elif i == n_steps - 1:
        axes[i].set_title('8', fontsize=13, fontweight='bold', color='#F44336')
    else:
        axes[i].set_title(f'{alphas[i]:.2f}', fontsize=9, color='gray')

fig.suptitle('Latent Space Interpolation: 3 $\\rightarrow$ 8', fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

print("Notice the smooth, gradual transition -- no jumps, no garbage.")
print("The 3 grows a top loop, then fills in to become an 8.")

In [ ]:
#@title 🎧 Listen: Interpolation Explanation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_23_interpolation_explanation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


This result is genuinely beautiful. Watch the progression carefully:

- The **3** starts with its two open curves
- Gradually, the top curve begins to close
- The shape fills in, gaining a rounder body
- By the end, it has become a full **8**

Every intermediate image is a plausible digit -- there are no sudden switches, no nonsense, no noise artifacts. This tells us that the VAE has learned a *continuous manifold* in latent space, where nearby points produce similar images.

**Why would this fail with a regular autoencoder?** In a regular autoencoder, the 3 and the 8 might be encoded as isolated points with nothing meaningful in between. Walking along the line connecting them would pass through dead zones -- regions the decoder has never seen during training -- producing garbage output.

In [ ]:
#@title 🎧 Listen: Manifold Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_24_manifold_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 The Digit Manifold

This is the crown jewel of VAE visualization. We are going to create a systematic grid over the entire 2D latent space, decode every single grid point, and assemble the results into one large composite image.

The result is a *map of all possible digits* that the decoder can produce.

We will sample a 15x15 grid spanning $[-3, 3]$ in both dimensions. Why $[-3, 3]$? Because the KL term encourages the latent distribution to be close to $\mathcal{N}(0, I)$, which means most of the probability mass lives within about 3 standard deviations of the origin.

In [ ]:
#@title 🎧 What to Look For: Manifold Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_25_manifold_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Create the digit manifold
model.eval()
n = 15  # 15x15 grid
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))

# Grid of z values spanning [-3, 3] in both dimensions
# We use the inverse CDF (ppf) of the normal distribution for even spacing
# in probability space, but linear spacing works well too
grid_x = np.linspace(-3, 3, n)
grid_y = np.linspace(3, -3, n)  # Flip y so top of image = high z2

with torch.no_grad():
    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z = torch.tensor([[xi, yi]], dtype=torch.float32).to(device)
            decoded = model.decoder(z).cpu().view(28, 28).numpy()
            figure[i * digit_size:(i + 1) * digit_size,
                   j * digit_size:(j + 1) * digit_size] = decoded

# Display
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(figure, cmap='gray_r', extent=[-3, 3, -3, 3])
ax.set_xlabel('$z_1$', fontsize=16)
ax.set_ylabel('$z_2$', fontsize=16)
ax.set_title('The Digit Manifold — Every Point in Latent Space Decoded',
             fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.15, color='white')

plt.tight_layout()
plt.show()

print(f"This image contains {n*n} = {n*n} decoded digits arranged on a {n}x{n} grid.")
print("Each position corresponds to a specific (z1, z2) coordinate in latent space.")

In [ ]:
#@title 🎧 Listen: Manifold Explanation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_26_manifold_explanation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Spend a moment studying this manifold. It is one of the most revealing visualizations in generative modeling.

**What you should notice:**

- **Different digits occupy different regions.** You can find 0s, 1s, 7s, and other digit classes concentrated in specific areas of the grid.
- **Smooth transitions.** As you move your eyes across the image, digit shapes morph gradually into each other. A 1 might slowly widen and curve until it becomes a 7, then a 9.
- **The center is densely populated.** Near $(0, 0)$, the digits tend to be more "average-looking" -- the decoder is hedging across multiple possibilities.
- **The edges get sparser.** At the fringes ($|z| > 2.5$), the decoder produces less confident outputs, because very few training images were encoded this far from the origin.

This manifold demonstrates that the VAE has learned a *continuous, structured* representation where the geometry of the latent space mirrors the visual similarity of the digits.

In [ ]:
#@title 🎧 Listen: Blurry Images Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_27_blurry_images_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 Why Are the Images Blurry?

You may have noticed that our generated and reconstructed digits look softer and less crisp than the original MNIST images. This is not a bug -- it is a fundamental property of how VAEs work.

Let us see this clearly by placing a sharp original next to its VAE reconstruction.

In [ ]:
#@title 🎧 What to Look For: Blurry Images Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_28_blurry_images_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Compare sharp originals vs. blurry reconstructions
model.eval()
fig, axes = plt.subplots(2, 6, figsize=(14, 5))

with torch.no_grad():
    sample_data, sample_labels = next(iter(test_loader))
    sample_data_flat = sample_data[:6].view(-1, 784).to(device)
    recon, _, _ = model(sample_data_flat)

for i in range(6):
    # Original (sharp)
    axes[0, i].imshow(sample_data[i].squeeze(), cmap='gray')
    axes[0, i].axis('off')
    axes[0, i].set_title(f'Digit {sample_labels[i].item()}', fontsize=11)

    # Reconstruction (blurry)
    axes[1, i].imshow(recon[i].cpu().view(28, 28), cmap='gray')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Original\n(sharp)', fontsize=12, rotation=0, labelpad=55, va='center')
axes[1, 0].set_ylabel('VAE\n(blurry)', fontsize=12, rotation=0, labelpad=55, va='center')

fig.suptitle('Why VAEs Produce Blurry Images', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Blurry Images Explanation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_29_blurry_images_explanation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Why does this happen?**

The culprit is the **pixel-independence assumption** in the decoder. Our decoder outputs a Bernoulli probability for each pixel independently. This means the model cannot express correlations between pixels -- it must make each pixel prediction on its own.

Consider a specific pixel that is black in 60% of training images of a "7" and white in the other 40%. What should the decoder predict?

- If it predicts **black** (probability 0.9), it will be wrong 40% of the time.
- If it predicts **white** (probability 0.9), it will be wrong 60% of the time.
- The optimal strategy is to predict a **middle value** (probability ~0.6) -- hedging its bets.

This averaging over modes produces the characteristic grayish, soft look of VAE outputs. The model literally predicts the *mean* of all possible pixel values, rather than committing to a sharp black-or-white image.

This limitation motivated the development of **diffusion models**, which use a fundamentally different approach (iterative denoising) and can generate much sharper images. But the *conceptual foundation* of VAEs -- learning a latent space and sampling from it -- carries forward into virtually every modern generative model.

In [ ]:
#@title 🎧 Before You Start: Todo Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_30_todo_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn -- TODO Sections

Now it is your turn. The two exercises below will deepen your understanding of how latent space operations work.

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_31_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 1: Implement the Interpolation Function

Write a function that takes two latent codes and returns a sequence of interpolated codes between them. This is the engine behind the filmstrip visualization we saw in Section 4.3.

In [ ]:
#@title 🎧 Before You Start: Todo1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_32_todo1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def interpolate_latent(z_start, z_end, n_steps=10):
    """
    Linearly interpolate between two latent codes.

    Args:
        z_start: Starting latent code, shape (1, latent_dim) or (latent_dim,)
        z_end: Ending latent code, shape (1, latent_dim) or (latent_dim,)
        n_steps: Number of interpolation steps (including endpoints)

    Returns:
        Tensor of shape (n_steps, latent_dim) with interpolated codes
    """
    # Ensure 1D
    z_start = z_start.view(-1)
    z_end = z_end.view(-1)

    # ============================================================
    # TODO: Create a tensor of n_steps evenly spaced points
    # between z_start and z_end (inclusive).
    #
    # Hint: Use torch.linspace to create interpolation weights
    # alpha = 0, 1/(n_steps-1), 2/(n_steps-1), ..., 1
    # Then compute z(alpha) = (1 - alpha) * z_start + alpha * z_end
    # ============================================================

    interpolated = None  # Replace with your implementation

    # ============================================================
    # END TODO
    # ============================================================

    return interpolated

In [ ]:
#@title 🎧 Before You Start: Todo1 Verification
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_33_todo1_verification.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Before You Start: Todo2 Verification
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_36_todo2_verification.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# --- Verification ---
# Test with known values
z_a = torch.tensor([0.0, 0.0])
z_b = torch.tensor([4.0, 2.0])
result = interpolate_latent(z_a, z_b, n_steps=5)

print("Testing interpolation from [0, 0] to [4, 2] with 5 steps:")
if result is not None:
    print(f"  Shape: {result.shape} (expected: torch.Size([5, 2]))")
    print(f"  First point:  {result[0].tolist()} (expected: [0.0, 0.0])")
    print(f"  Middle point: {result[2].tolist()} (expected: [2.0, 1.0])")
    print(f"  Last point:   {result[4].tolist()} (expected: [4.0, 2.0])")

    # Visualize: interpolate between a 1 and a 0
    model.eval()
    digit_1_img = None
    digit_0_img = None
    for data, labels in test_loader:
        for i in range(len(labels)):
            if labels[i] == 1 and digit_1_img is None:
                digit_1_img = data[i]
            if labels[i] == 0 and digit_0_img is None:
                digit_0_img = data[i]
            if digit_1_img is not None and digit_0_img is not None:
                break
        if digit_1_img is not None and digit_0_img is not None:
            break

    with torch.no_grad():
        mu_1, _ = model.encoder(digit_1_img.view(1, 784).to(device))
        mu_0, _ = model.encoder(digit_0_img.view(1, 784).to(device))
        z_path = interpolate_latent(mu_1.cpu(), mu_0.cpu(), n_steps=10).to(device)
        decoded_path = model.decoder(z_path).cpu().view(-1, 28, 28)

    fig, axes = plt.subplots(1, 10, figsize=(16, 2.2))
    for i in range(10):
        axes[i].imshow(decoded_path[i], cmap='gray')
        axes[i].axis('off')
    axes[0].set_title('1', fontweight='bold', color='#2196F3')
    axes[-1].set_title('0', fontweight='bold', color='#F44336')
    fig.suptitle('Your Interpolation: 1 $\\rightarrow$ 0', fontsize=13, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    print("If you see a smooth transition from 1 to 0, your implementation is correct!")
else:
    print("  interpolated is None -- implement the TODO above!")

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_34_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Generate the Digit Manifold Grid

Write a function that creates the digit manifold visualization. Given the decoder, a grid size, and a range, it should decode every point on the grid and assemble the results into one large image.

In [ ]:
#@title 🎧 Before You Start: Todo2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_35_todo2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def generate_manifold(decoder, n=20, z_range=3.0, device='cpu'):
    """
    Generate a digit manifold by decoding a grid of latent codes.

    Args:
        decoder: Trained decoder network
        n: Grid size (n x n points)
        z_range: Range of z values, grid spans [-z_range, z_range]
        device: torch device

    Returns:
        numpy array of shape (n*28, n*28) containing the assembled image
    """
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))

    # Create grid coordinates
    grid_x = np.linspace(-z_range, z_range, n)
    grid_y = np.linspace(z_range, -z_range, n)  # Flip y-axis

    # ============================================================
    # TODO: Fill in the figure array by decoding each grid point.
    #
    # For each (i, j) position in the grid:
    #   1. Create a latent code z = [grid_x[j], grid_y[i]]
    #   2. Pass z through the decoder to get a 784-dim output
    #   3. Reshape to 28x28 and place it in the correct position
    #      in the figure array
    #
    # Hint: Use torch.no_grad() to avoid unnecessary computation.
    # The decoded image goes at:
    #   figure[i*28:(i+1)*28, j*28:(j+1)*28]
    # ============================================================

    # Your code here

    # ============================================================
    # END TODO
    # ============================================================

    return figure

In [ ]:
# --- Verification ---
manifold = generate_manifold(model.decoder, n=20, z_range=3.0, device=device)

if manifold.max() > 0:
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(manifold, cmap='gray_r', extent=[-3, 3, -3, 3])
    ax.set_xlabel('$z_1$', fontsize=16)
    ax.set_ylabel('$z_2$', fontsize=16)
    ax.set_title('Your Digit Manifold (20x20 Grid)', fontsize=15, fontweight='bold')
    ax.grid(True, alpha=0.15, color='white')
    plt.tight_layout()
    plt.show()
    print(f"Manifold generated: {manifold.shape[0]}x{manifold.shape[1]} pixels")
    print(f"That is {20*20} = 400 decoded digits arranged in a grid!")
    print("If you see smooth transitions across the grid, your implementation is correct!")
else:
    print("Manifold is all zeros -- implement the TODO above!")

In [ ]:
#@title 🎧 Narration: Gallery Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_37_gallery_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together -- The VAE Gallery

Let us create a single composite figure that showcases everything we have explored: the latent space scatter plot, generated samples, the interpolation filmstrip, and the digit manifold, all in one place.

In [ ]:
#@title 🎧 What to Look For: Gallery Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_38_gallery_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# VAE Gallery: everything in one figure
model.eval()

fig = plt.figure(figsize=(18, 14))

# ---- Panel 1: Latent Space Scatter ----
ax1 = fig.add_subplot(2, 2, 1)
scatter = ax1.scatter(all_mu[:, 0], all_mu[:, 1], c=all_labels, cmap='tab10',
                      s=2, alpha=0.4, edgecolors='none')
cbar = plt.colorbar(scatter, ax=ax1, ticks=range(10), shrink=0.8)
cbar.set_label('Digit')
ax1.set_xlabel('$z_1$')
ax1.set_ylabel('$z_2$')
ax1.set_title('(a) Latent Space', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# ---- Panel 2: Generated Samples ----
ax2 = fig.add_subplot(2, 2, 2)
with torch.no_grad():
    z_grid = torch.randn(36, 2).to(device)
    gen_grid = model.decoder(z_grid).cpu().view(-1, 28, 28)

grid_img = np.zeros((6 * 28, 6 * 28))
for i in range(6):
    for j in range(6):
        grid_img[i*28:(i+1)*28, j*28:(j+1)*28] = gen_grid[i*6+j].numpy()
ax2.imshow(grid_img, cmap='gray')
ax2.axis('off')
ax2.set_title('(b) Generated Samples', fontsize=13, fontweight='bold')

# ---- Panel 3: Interpolation Filmstrip ----
ax3 = fig.add_subplot(2, 2, 3)
with torch.no_grad():
    alphas_gallery = torch.linspace(0, 1, 10)
    z_interp_gallery = torch.stack([
        (1 - a) * mu_3 + a * mu_8 for a in alphas_gallery
    ]).squeeze(1).to(device)
    interp_gallery = model.decoder(z_interp_gallery).cpu().view(-1, 28, 28)

strip = np.concatenate([interp_gallery[i].numpy() for i in range(10)], axis=1)
ax3.imshow(strip, cmap='gray')
ax3.axis('off')
ax3.set_title('(c) Interpolation: 3 $\\rightarrow$ 8', fontsize=13, fontweight='bold')

# ---- Panel 4: Digit Manifold ----
ax4 = fig.add_subplot(2, 2, 4)
n_gallery = 12
digit_size = 28
manifold_gallery = np.zeros((digit_size * n_gallery, digit_size * n_gallery))
grid_x_g = np.linspace(-3, 3, n_gallery)
grid_y_g = np.linspace(3, -3, n_gallery)

with torch.no_grad():
    for i, yi in enumerate(grid_y_g):
        for j, xi in enumerate(grid_x_g):
            z = torch.tensor([[xi, yi]], dtype=torch.float32).to(device)
            decoded = model.decoder(z).cpu().view(28, 28).numpy()
            manifold_gallery[i*digit_size:(i+1)*digit_size,
                             j*digit_size:(j+1)*digit_size] = decoded

ax4.imshow(manifold_gallery, cmap='gray_r')
ax4.axis('off')
ax4.set_title('(d) Digit Manifold', fontsize=13, fontweight='bold')

fig.suptitle('The VAE Gallery -- Everything Our Model Has Learned',
             fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

print("Four views of the same model, four windows into the latent space.")

In [ ]:
#@title 🎧 Transition: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_39_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Final Output

Let us produce the two signature outputs of this notebook series: a high-resolution digit manifold and a multi-pair interpolation showcase.

In [ ]:
#@title 🎧 What to Look For: Final Manifold Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_40_final_manifold_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Final Output 1: High-resolution digit manifold
model.eval()
n_final = 20
digit_size = 28
manifold_final = np.zeros((digit_size * n_final, digit_size * n_final))
grid_x_f = np.linspace(-3, 3, n_final)
grid_y_f = np.linspace(3, -3, n_final)

with torch.no_grad():
    for i, yi in enumerate(grid_y_f):
        for j, xi in enumerate(grid_x_f):
            z = torch.tensor([[xi, yi]], dtype=torch.float32).to(device)
            decoded = model.decoder(z).cpu().view(28, 28).numpy()
            manifold_final[i*digit_size:(i+1)*digit_size,
                           j*digit_size:(j+1)*digit_size] = decoded

fig, ax = plt.subplots(figsize=(14, 14))
ax.imshow(manifold_final, cmap='gray_r', extent=[-3, 3, -3, 3])
ax.set_xlabel('$z_1$', fontsize=18)
ax.set_ylabel('$z_2$', fontsize=18)
ax.set_title('The Complete Digit Manifold (20x20)',
             fontsize=17, fontweight='bold')
ax.tick_params(labelsize=13)
plt.tight_layout()
plt.show()

print(f"400 unique digits decoded from a uniform grid over the latent space.")

In [ ]:
#@title 🎧 What to Look For: Final Interpolation Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_41_final_interpolation_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Final Output 2: Multi-pair interpolation
model.eval()

# Find one example of each digit pair we want to interpolate
pairs = [(0, 1), (2, 7), (3, 8), (4, 9), (5, 6)]
pair_images = {}

for data, labels in test_loader:
    for i in range(len(labels)):
        lab = labels[i].item()
        if lab not in pair_images:
            pair_images[lab] = data[i]
    if len(pair_images) == 10:
        break

fig, axes = plt.subplots(len(pairs), 12, figsize=(18, 7.5))

for row, (d1, d2) in enumerate(pairs):
    with torch.no_grad():
        mu_a, _ = model.encoder(pair_images[d1].view(1, 784).to(device))
        mu_b, _ = model.encoder(pair_images[d2].view(1, 784).to(device))

        alphas_final = torch.linspace(0, 1, 12)
        z_path = torch.stack([
            (1 - a) * mu_a + a * mu_b for a in alphas_final
        ]).squeeze(1).to(device)
        decoded_path = model.decoder(z_path).cpu().view(-1, 28, 28)

    for col in range(12):
        axes[row, col].imshow(decoded_path[col], cmap='gray')
        axes[row, col].axis('off')

    axes[row, 0].set_ylabel(f'{d1}$\\rightarrow${d2}', fontsize=12,
                             rotation=0, labelpad=35, va='center')

fig.suptitle('Smooth Interpolations Between Digit Pairs',
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("Every row shows a smooth morphing between two different digits.")
print("No jumps, no garbage, no dead zones -- just continuous transformation.")

In [ ]:
#@title 🎧 Listen: Congratulations
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_42_congratulations.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
print()
print("=" * 65)
print("  Congratulations! You have built a Variational Autoencoder")
print("  completely from scratch!")
print("=" * 65)
print()
print("  From latent variables to the ELBO to a working generative")
print("  model -- you now understand one of the most elegant")
print("  frameworks in machine learning.")
print()
print("  Key insight: generation = learning a latent space")
print("                            + sampling from it")
print("=" * 65)

In [ ]:
#@title 🎧 Transition: Reflection Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_43_reflection_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Reflection and Next Steps

We have come a long way. Let us reflect on what we accomplished across all four notebooks:

1. **Notebook 1**: We built the intuition -- latent variables as hidden recipes, the encoder-decoder framework, and why making it probabilistic matters.
2. **Notebook 2**: We derived the math -- the ELBO, the KL divergence closed form, and the reparameterization trick.
3. **Notebook 3**: We implemented everything in PyTorch and trained a working VAE on MNIST.
4. **Notebook 4 (this one)**: We explored the latent space and saw the payoff -- structured representations, generation, interpolation, and the digit manifold.

In [ ]:
#@title 🎧 Listen: Reflection Questions
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_44_reflection_questions.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Reflection Questions

Take a few minutes to think about these:

1. **Why does the manifold look different at the center vs. the edges?** What role does the KL divergence play in shaping this distribution?

2. **If we increased the latent dimension from 2 to 20**, what would change? The reconstructions would improve (more capacity), but we could no longer make scatter plots or 2D manifolds. What is the trade-off between latent dimension and interpretability?

3. **Why does interpolation work in the VAE's latent space but not in pixel space?** If you averaged the pixel values of a 3 and an 8 directly (without encoding), what would you get? Try it and compare.

In [ ]:
#@title 🎧 Listen: Optional Challenges
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_45_optional_challenges.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Optional Challenges

If you want to push further, here are some exciting directions:

- **Try a higher latent dimension** (e.g., 10 or 20). Train the model and compare reconstruction quality. You will not be able to visualize the latent space as a 2D scatter plot, but you can use t-SNE or UMAP to project it down to 2D for visualization.

- **Add convolutional layers.** Replace the fully-connected encoder and decoder with convolutional and transposed-convolutional layers. This should improve image quality noticeably while keeping the same ELBO training objective.

- **Try Fashion-MNIST or CIFAR-10.** Fashion-MNIST is a drop-in replacement for MNIST but with clothing items instead of digits. CIFAR-10 (32x32 color images) is significantly harder and will expose the blurriness limitation more clearly.

- **Implement a conditional VAE (CVAE).** Pass the digit label as an additional input to both the encoder and decoder. This lets you control *which* digit to generate while the latent code controls the *style*.

In [ ]:
#@title 🎧 Listen: Bigger Picture
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_46_bigger_picture.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### The Bigger Picture

The blurriness of VAE outputs is not a failure -- it is a *design trade-off*. By using independent pixel distributions, the model gains a tractable training objective (the ELBO) and a beautifully structured latent space. The cost is soft, averaged outputs.

This trade-off directly motivated the development of **diffusion models**, which bypass the encoder-decoder bottleneck entirely and instead learn to iteratively denoise random noise into sharp images. But at their core, diffusion models still rely on the same fundamental insight that VAEs taught us:

**Generation = learning a structured representation of data + sampling from it.**

This idea is the thread that connects VAEs, GANs, diffusion models, normalizing flows, and even large language models. You now understand one of the most foundational frameworks in all of generative AI.

Thank you for working through this series. We hope you enjoyed the journey.

---

**References:**
- Kingma & Welling, "Auto-Encoding Variational Bayes" (2013)
- Doersch, "Tutorial on Variational Autoencoders" (2016)
- Blei, Kucukelbir & McAuliffe, "Variational Inference: A Review for Statisticians" (2017)